#### MNIST AlexNet

In [1]:
# import...

from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [3]:
# Load the data
mnist = input_data.read_data_sets('/tmp/data/', one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
# Set parameters
learning_rate = 0.001
training_iters = 300000
batch_size = 64
display_step = 100
n_input = 784
n_classes = 10
dropout = 0.8

In [6]:
# tensorflow placeholder
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

In [28]:
# conv2d
def conv2d(name, l_input, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(l_input, w, strides=[1,1,1,1], padding='SAME'),b), name=name)

In [29]:
# max_pool
def max_pool(name, l_input, k):
    return tf.nn.max_pool(l_input, ksize=[1,k,k,1], strides=[1,k,k,1], padding='SAME', name=name)

In [30]:
# local response normalization
def norm(name, l_input, lsize=4):    

    # alpha: scale factor, beta: exponential value
    return tf.nn.lrn(l_input, lsize, bias=1.0, alpha=0.001/9.0, beta=0.75, name=name)

In [25]:
# Set Parameters
weights = {'wc1': tf.Variable(tf.random_normal([3,3,1,64])),
          'wc2': tf.Variable(tf.random_normal([3,3,64,128])),
          'wc3': tf.Variable(tf.random_normal([3,3,128,256])),
          'wd1': tf.Variable(tf.random_normal([4*4*256, 1024])), # batch_size(64)*1024
          'wd2': tf.Variable(tf.random_normal([1024, 1024])), # batch_size*1024 matmul 1024x1024
          'out': tf.Variable(tf.random_normal([1024,10]))}  # batch_size(64)x1024 = 내적 = 1024x10

biases = {'bc1': tf.Variable(tf.random_normal([64])),
         'bc2': tf.Variable(tf.random_normal([128])),
         'bc3': tf.Variable(tf.random_normal([256])),
         'bd1': tf.Variable(tf.random_normal([1024])),
         'bd2': tf.Variable(tf.random_normal([1024])),
         'out': tf.Variable(tf.random_normal([n_classes]))}

In [31]:
# alexnet
def alex_net(_X, _weights, _biases, _dropout):
    
    # Conv 1 layer
    # 1 = black & number of output
    _X = tf.reshape(_X, shape=[-1,28,28,1])
    
    # weights = # of filters = # of outputs
    conv1 = conv2d('conv1', _X, _weights['wc1'], _biases['bc1'])
    
    # pooling 2x2 (0.25 size)
    pool1 = max_pool('pool1', conv1, k=2)
    
    # norm: data/size of vector -> vector=1 fix, find angle
    norm1 = norm('norm1', pool1, lsize=4)
    
    # dropout: reduce overfitting
    norm1 = tf.nn.dropout(norm1, _dropout)
    ###############################################################
    # Conv 2 layer
    conv2 = conv2d('conv2', norm1, _weights['wc2'], _biases['bc2'])
    pool2 = max_pool('pool2',conv2, k=2)
    norm2 = norm('norm2',pool2, lsize=4)
    norm2 = tf.nn.dropout(norm2, _dropout)
    ###############################################################
    # Conv 3 layer
    conv3 = conv2d('conv3',norm2, _weights['wc3'], _biases['bc3'])
    pool3 = max_pool('pool3', conv3, k=2)
    norm3 = norm('norm3', pool3, lsize=4)
    norm3 = tf.nn.dropout(norm3, _dropout)
    ###############################################################
    # FFNN
    # Step1. flattening
    dense1 = tf.reshape(norm3, [-1, _weights['wd1'].get_shape().as_list()[0]])
    
    # Step2. Dense (matmul -> bias)
    dense1 = tf.nn.relu(tf.matmul(dense1, _weights['wd1']) + _biases['bd1'], name='fc1')
    dense2 = tf.nn.relu(tf.matmul(dense1, _weights['wd2']) + _biases['bd2'], name='fc2')
    
    # Step3. Output
    out = tf.matmul(dense2, _weights['out']) + _biases['out']
    
    return out

In [33]:
# prediction
pred = alex_net(x, weights, biases, keep_prob)

W0923 20:01:29.280263 37356 deprecation.py:506] From <ipython-input-31-2e15833a4341>:18: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [34]:
# Calculate mean(prediction - label)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))

In [38]:
# Train = Optimizer (learning rate)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

In [40]:
# Accuracy
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
# Session
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    step = 1
    
    while step*batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys, keep_prob: dropout})
        
        if step%display_step == 0:
            acc = sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            loss = sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            print('Iter' + str(step*batch_size) + ', Minibatch Loss=' + '{:.6f}'.format(loss) +', Training Accuracy=' + '{:.5f}'.format(acc))
        
        step += 1
    print('Optimization Finished!')
    print('Testing Accuracy:', sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                                            y: mnist.test.labels[:256],
                                                            keep_prob: 1.}))

Iter6400, Minibatch Loss=68513.718750, Training Accuracy=0.39062
Iter12800, Minibatch Loss=23020.757812, Training Accuracy=0.71875
Iter19200, Minibatch Loss=22225.437500, Training Accuracy=0.70312
Iter25600, Minibatch Loss=13790.101562, Training Accuracy=0.82812
Iter32000, Minibatch Loss=7043.297852, Training Accuracy=0.82812
Iter38400, Minibatch Loss=13035.176758, Training Accuracy=0.76562
Iter44800, Minibatch Loss=8624.713867, Training Accuracy=0.90625
Iter51200, Minibatch Loss=11614.361328, Training Accuracy=0.78125
